In [1]:
import numpy as np
import pandas as pd

In [2]:
df_train = pd.read_csv('train.csv', encoding='utf-8',nrows=200000)
df_train['id'] = df_train['id'].apply(str)

In [3]:
df_all = df_train
df_all['question1'].fillna('', inplace=True)
df_all['question2'].fillna('', inplace=True)

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
import itertools


In [5]:
counts_vectorizer = CountVectorizer(max_features=10000-1).fit(
    itertools.chain(df_all['question1'], df_all['question2']))
other_index = len(counts_vectorizer.vocabulary_)


In [6]:
print(other_index)
print(counts_vectorizer.vocabulary_)

9999
{'tonight': 9115, 'edm': 3007, 'exercising': 3298, 'waiting': 9671, 'cyrus': 2402, 'pretend': 6965, 'anniversary': 614, 'property': 7084, 'turning': 9294, 'purifier': 7182, 'day': 2449, 'manner': 5512, '90': 218, 'ese': 3213, 'reads': 7343, 'cruel': 2325, 'courses': 2243, 'nose': 6165, 'con': 2032, 'writes': 9921, 'scams': 7880, 'dba': 2451, 'stretch': 8610, '2008': 78, 'poker': 6808, 'justin': 4970, 'ui': 9342, 'brushing': 1403, 'change': 1673, 'wwii': 9932, 'balloons': 975, 'served': 8053, 'overcome': 6422, 'advocate': 404, 'shah': 8085, 'ranks': 7297, 'stronger': 8621, 'experiences': 3325, 'outgoing': 6411, 'although': 539, 'biblical': 1158, 'horny': 4317, 'cashless': 1587, 'rationale': 7317, '30': 117, 'plant': 6757, 'hearing': 4172, 'trained': 9186, 'borderline': 1294, 'boredom': 1297, 'hitler': 4260, 'paradox': 6496, 'verizon': 9550, 'buffett': 1421, 'newspapers': 6100, 'length': 5217, 'constipation': 2099, 'invested': 4765, 'simulation': 8215, 'afterlife': 429, 'shock': 813

In [7]:
print(df_all['question1'].shape)

(200000,)


In [8]:
import re
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [9]:
print(counts_vectorizer.token_pattern)
print(counts_vectorizer)

(?u)\b\w\w+\b
CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=9999, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)


In [10]:


words_tokenizer = re.compile(counts_vectorizer.token_pattern)
#print(words_tokenizer)
t=df_all[0:1]['question1']
print(t)



0    What is the step by step guide to invest in sh...
Name: question1, dtype: object


In [11]:
def create_padded_seqs(texts, max_len=10):
    seqs = texts.apply(lambda s: 
        [counts_vectorizer.vocabulary_[w] if w in counts_vectorizer.vocabulary_ else other_index
         for w in words_tokenizer.findall(s.lower())])
    print(seqs)
    print(len(seqs))
    print(seqs.shape)
    return pad_sequences(seqs, maxlen=max_len)

In [12]:
sample=create_padded_seqs(t)
print(sample)

0    [9786, 4815, 8983, 8558, 1470, 8558, 4033, 909...
Name: question1, dtype: object
1
(1,)
[[1470 8558 4033 9093 4764 4531 8097 5541 4531 4563]]


In [13]:
X1_train, X1_val, X2_train, X2_val, y_train, y_val = \
    train_test_split(create_padded_seqs(df_all[df_all['id'].notnull()]['question1']), 
                     create_padded_seqs(df_all[df_all['id'].notnull()]['question2']),
                     df_all[df_all['id'].notnull()]['is_duplicate'].values,
                     stratify=df_all[df_all['id'].notnull()]['is_duplicate'].values,
                     test_size=0.3, random_state=1989)

0         [9786, 4815, 8983, 8558, 1470, 8558, 4033, 909...
1         [9786, 4815, 8983, 8584, 6263, 9999, 9999, 999...
2         [4345, 1520, 4552, 8983, 8433, 6263, 5984, 471...
3         [9814, 547, 5683, 9559, 5361, 4345, 1520, 8356...
4         [9800, 6306, 2768, 4531, 9721, 9999, 8684, 782...
5         [817, 547, 1554, 8702, 1541, 5885, 591, 1541, ...
6                                        [8155, 1465, 9999]
7                            [4345, 1520, 1047, 3937, 9999]
8                      [9794, 2801, 9968, 9456, 4665, 6263]
9          [5917, 1981, 1520, 4061, 5984, 1695, 9999, 9999]
10        [5720, 9093, 3543, 8042, 6263, 9999, 9464, 999...
11          [4345, 2801, 7339, 591, 3543, 5984, 9976, 1960]
12               [9786, 1520, 5475, 6697, 2973, 9093, 5185]
13               [9786, 9708, 9972, 3568, 8077, 3323, 5274]
14        [9786, 717, 8983, 5158, 9093, 1673, 9972, 8543...
15        [9786, 9911, 9270, 6958, 5623, 3651, 2371, 471...
16                                 [9786

In [14]:
import keras.layers as lyr
from keras.models import Model


In [15]:
print(X1_train.shape)
print(X2_train.shape)
print(X1_val.shape)
print(X2_val.shape)
print(len(y_train))
print(X1_train.shape[1:])
print(X1_train.max())


(140000, 10)
(140000, 10)
(60000, 10)
(60000, 10)
140000
(10,)
9999


In [16]:
input1_tensor = lyr.Input(X1_train.shape[1:])
input2_tensor = lyr.Input(X2_train.shape[1:])

words_embedding_layer = lyr.Embedding(X1_train.max() + 1, 100)
seq_embedding_layer = lyr.LSTM(256, activation='tanh')

seq_embedding = lambda tensor: seq_embedding_layer(words_embedding_layer(tensor))


In [17]:
merge_layer = lyr.multiply([seq_embedding(input1_tensor), seq_embedding(input2_tensor)])

In [18]:

dense1_layer = lyr.Dense(16, activation='sigmoid')(merge_layer)
ouput_layer = lyr.Dense(1, activation='sigmoid')(dense1_layer)
model = Model([input1_tensor, input2_tensor], ouput_layer)

model.compile(loss='binary_crossentropy', optimizer='adam')
model.summary()



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 10)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 10)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 10, 100)      1000000     input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 256)          365568      embedding_1[0][0]                
          

In [20]:
model.fit([X1_train, X2_train], y_train, 
          batch_size=128, epochs=6, verbose=2)

Epoch 1/6
 - 250s - loss: 0.4953
Epoch 2/6
 - 246s - loss: 0.4397
Epoch 3/6
 - 246s - loss: 0.3926
Epoch 4/6
 - 268s - loss: 0.3475
Epoch 5/6
 - 250s - loss: 0.3040
Epoch 6/6
 - 247s - loss: 0.2630


In [22]:
ans=model.predict([X1_val,X2_val],batch_size=128)


In [24]:
print(ans)
print(len(ans))

[[0.29347536]
 [0.3206907 ]
 [0.9031487 ]
 ...
 [0.0379034 ]
 [0.06468935]
 [0.52029973]]
60000


In [28]:
rr=np.around(ans)
print(rr)


[[0.]
 [0.]
 [1.]
 ...
 [0.]
 [0.]
 [1.]]


In [37]:
x=[]
for word in rr:
    x.append(word[0])
#print(x)
print(len(x))
print(x[0:3])

60000
[0.0, 0.0, 1.0]


In [39]:
print(y_val)
yy=y_val.tolist()
print(yy)

[1 1 0 ... 1 0 1]
[1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 

In [45]:
count=0
for i in range(0,len(yy)):
    if x[i]==yy[i]:
        count+=1
print(count)

47088


In [51]:
print("Accuracy",count/len(yy))
#print(len(yy))

Accuracy 0.7848
